In [1]:
import sys

manipulate_path = '/home/jiechu/Data/TinyRobotBench/manipulate'

if manipulate_path not in sys.path:
    sys.path.insert(0, manipulate_path)

print(sys.path)

['/home/jiechu/Data/TinyRobotBench/manipulate', '/home/jiechu/Data/TinyRobotBench/manipulate/tests', '/home/jiechu/miniconda3/envs/roboSim/lib/python39.zip', '/home/jiechu/miniconda3/envs/roboSim/lib/python3.9', '/home/jiechu/miniconda3/envs/roboSim/lib/python3.9/lib-dynload', '', '/home/jiechu/miniconda3/envs/roboSim/lib/python3.9/site-packages']


In [2]:
import sapien.core as sapien
from sapien.core import Actor
from sapien.utils import Viewer
import numpy as np
from transforms3d.euler import euler2quat, euler2mat, mat2euler
from PIL import Image, ImageColor
import matplotlib.pyplot as plt
from IPython.display import display

In [3]:
from utils import *
from perception import *
from config import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
#---------------------------------------------------------
# Initialize Basic Scene
#---------------------------------------------------------
engine = sapien.Engine()
renderer = sapien.SapienRenderer()
engine.set_renderer(renderer)

scene = engine.create_scene()
scene.set_timestep(1 / 100.0)

scene.add_ground(-1)
physical_material = scene.create_physical_material(static_friction=1, dynamic_friction=1, restitution=0.0)
scene.default_physical_material = physical_material


scene.set_ambient_light(color=[0.5, 0.5, 0.5])
scene.add_directional_light(direction=[0, 1, -1], color=[0.5, 0.5, 0.5], shadow=True)
scene.add_point_light(position=[1, 2, 2], color=[1, 1, 1], shadow=True)
scene.add_point_light(position=[1, -2, 2], color=[1, 1, 1], shadow=True)
scene.add_point_light(position=[-1, 0, 1], color=[1, 1, 1], shadow=True)


# #---------------------------------------------------------
# # Create Tabletop
# #---------------------------------------------------------
# table top
table=create_table(
    scene=scene,
    pose=sapien.Pose([0.56, 0, 0]),
    size=1.0,
    height=1,
    thickness=0.1,
    name="table",
    )
#objects
box = create_box(
    scene,
    sapien.Pose(p=[0.56, 0, 0.02], q=euler2quat(0, 0, np.pi/2)),
    half_size=[0.02, 0.05, 0.02],
    color=[1., 0., 0.],
    name='box',
)
sphere = create_sphere(
    scene,
    sapien.Pose(p=[-0.3+0.56, -0.4, 0.02]),
    radius=0.02,
    color=[0., 1., 0.],
    name='sphere',
)
capsule = create_capsule(
    scene,
    sapien.Pose(p=[0.3+0.3, 0.2, 0.02]),
    radius=0.02,
    half_length=0.03,
    color=[0., 0., 1.],
    name='capsule',
)
banana = load_object_mesh(
    scene, 
    sapien.Pose(p=[-0.2+0.56, 0, 0.01865]), 
    collision_file_path=manipulate_root_path+'assets/object/banana/collision_meshes/collision.obj',
    visual_file_path=manipulate_root_path+'assets/object/banana/visual_meshes/visual.dae',
    name='banana',
)


robot_urdf_path=manipulate_root_path+"assets/robot/panda/panda.urdf"
robot_srdf_path=manipulate_root_path+"assets/robot/panda/panda.srdf"
move_group="panda_hand"
# Robot
# Load URDF
init_qpos=[0, 0.19634954084936207, 0.0, -2.617993877991494, 0.0, 2.941592653589793, 0.7853981633974483, 0, 0]
robot=load_robot(
    scene=scene,
    pose=sapien.Pose([0, 0, 0], [1, 0, 0, 0]),
    init_qpos=init_qpos,
    urdf_file_path=robot_urdf_path,
    name="panda_robot",
    )

# scene.remove_articulation(robot)

#---------------------------------------------------------
# Add a Camera
#---------------------------------------------------------
near, far = 0.05, 100
width, height = 640, 480
camera = scene.add_camera(
    name="camera",
    width=width,
    height=height,
    fovy=1,
    near=near,
    far=far,
)
camera.set_pose(sapien.Pose(p=[1.46, 0, 0.6], q=euler2quat(0, 0.8, np.pi)))

print('Intrinsic matrix\n', camera.get_intrinsic_matrix())

scene.step()  # make everything set
scene.update_render()
camera.take_picture()

# ---------------------------------------------------------------------------- #
# RGBA
# ---------------------------------------------------------------------------- #
rgba = camera.get_float_texture('Color')  # [H, W, 4]
# An alias is also provided
# rgba = camera.get_color_rgba()  # [H, W, 4]
rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
rgba_pil = Image.fromarray(rgba_img)
rgba_pil.save('color.png')


# ---------------------------------------------------------------------------- #
# Generate Point Cloud
# ---------------------------------------------------------------------------- #
position = camera.get_float_texture('Position')
points_opengl = position[..., :3][position[..., 3] < 1]
points_color = rgba[position[..., 3] < 1][..., :3]
model_matrix = camera.get_model_matrix()
points_world = points_opengl @ model_matrix[:3, :3].T + model_matrix[:3, 3]

#---------------------------------------------------------
# Hold on the GUI Window
#---------------------------------------------------------
viewer = Viewer(renderer)
viewer.set_scene(scene)
# We show how to set the viewer according to the pose of a camera
# opengl camera -> sapien world
model_matrix = camera.get_model_matrix()
# sapien camera -> sapien world
# You can also infer it from the camera pose
model_matrix = model_matrix[:, [2, 0, 1, 3]] * np.array([-1, -1, 1, 1])
# The rotation of the viewer camera is represented as [roll(x), pitch(-y), yaw(-z)]
rpy = mat2euler(model_matrix[:3, :3]) * np.array([1, -1, -1])
viewer.set_camera_xyz(*model_matrix[0:3, 3])
viewer.set_camera_rpy(*rpy)
viewer.window.set_camera_parameters(near=0.05, far=100, fovy=1)
viewer.toggle_axes(False)

# viewer.render()
camera.take_picture()
rgba = camera.get_float_texture('Color')  # [H, W, 4]
# An alias is also provided
# rgba = camera.get_color_rgba()  # [H, W, 4]
rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
rgba_pil = Image.fromarray(rgba_img)
rgba_pil.save('color.png')

seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]

num_colors = 100
color_palette = np.random.randint(0, 256, size=(num_colors, 3), dtype=np.uint8)
# colormap = sorted(set(ImageColor.colormap.values()), reverse=True)
# color_palette = np.array([ImageColor.getrgb(color) for color in colormap],
#                             dtype=np.uint8)
label0_image = seg_labels[..., 0].astype(np.uint8)  # mesh-level
label1_image = seg_labels[..., 1].astype(np.uint8)  # actor-level
# Or you can use aliases below
# label0_image = camera.get_visual_segmentation()
# label1_image = camera.get_actor_segmentation()
label0_pil = Image.fromarray(color_palette[label0_image])
label0_pil.save('label0.png')
label1_pil = Image.fromarray(color_palette[label1_image])
label1_pil.save('label1.png')

while not viewer.closed:
    scene.update_render()
    viewer.render()

Intrinsic matrix
 [[439.31705   0.      320.     ]
 [  0.      439.31705 240.     ]
 [  0.        0.        1.     ]]


In [6]:
def get_pcd_from_actor(actor: Actor):
    vis_body = actor.get_visual_bodies()[0]
    render_shape = vis_body.get_render_shapes()[0]
    vertices = np.unique(render_shape.mesh.vertices, axis=0) # deduplicate

    actor_type = actor.get_builder().get_visuals()[0].type
    if actor_type == "Box":
        vertices = vertices * actor.get_builder().get_visuals()[0].scale 
    elif actor_type == "Sphere":
        vertices = vertices * actor.get_builder().get_visuals()[0].radius
    elif actor_type == "Capsule" or actor_type == "File":
        vertices = vertices

    tf_mat=actor.get_pose().to_transformation_matrix()
    vertices_homo=np.concatenate((vertices, np.ones((vertices.shape[0],1))), axis=-1)
    pcd = (tf_mat@vertices_homo.T).T[:,:-1]
    
    return pcd

In [5]:
from perception.scene_graph import SceneGraph, Node
scenegraph=SceneGraph()

names=get_names_in_scene(scene=scene)[2:]

obj_entities=[]
for name in names:
    obj_entity = get_object_by_name(scene=scene, name=name)
    obj_entities.append(obj_entity)
    node=Node(obj_entity.name, get_pcd_from_actor(obj_entity))
    scenegraph.add_node_wo_state(node)

In [6]:
print(scenegraph)

  [Nodes]:
    box -- position: [0.56, 0.00, 0.02], x_range: [0.51, 0.61], y_range: [-0.02, 0.02], z_range: [-0.00, 0.04]
    sphere -- position: [0.26, -0.40, 0.02], x_range: [0.24, 0.28], y_range: [-0.42, -0.38], z_range: [0.00, 0.04]
    capsule -- position: [0.60, 0.20, 0.02], x_range: [0.55, 0.65], y_range: [0.18, 0.22], z_range: [-0.00, 0.04]
    banana -- position: [0.36, -0.01, 0.02], x_range: [0.26, 0.46], y_range: [-0.05, 0.02], z_range: [0.00, 0.04]
  [Edges]:



In [20]:
box_pcd = get_pcd_from_actor(box)
print(box_pcd)

[[ 6.09999591e-01 -1.99994209e-02  3.49144254e-06]
 [ 6.09999154e-01 -1.99945904e-02  4.00034905e-02]
 [ 5.09999578e-01 -1.99986222e-02  2.39706028e-06]
 [ 5.09999140e-01 -1.99937917e-02  4.00023962e-02]
 [ 6.09999911e-01  2.00005829e-02 -1.33907784e-06]
 [ 6.09999473e-01  2.00054135e-02  3.99986600e-02]
 [ 5.09999897e-01  2.00013816e-02 -2.43346009e-06]
 [ 5.09999460e-01  2.00062122e-02  3.99975656e-02]]


In [25]:
np.max(box_pcd, axis=0)[:2]-np.min(box_pcd, axis=0)[:2]

array([0.10000077, 0.04000563])

In [26]:
np.linalg.norm(np.max(box_pcd, axis=0)[:2]-np.min(box_pcd, axis=0)[:2])

0.10770610382668099